In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Olafsvik,IS,64.8945,-23.7142,27.19,55,100,23.47,overcast clouds
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,74.66,96,100,8.48,light rain
2,2,Boa Vista,BR,2.8197,-60.6733,80.58,61,40,9.22,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.26,74,16,15.68,few clouds
4,4,Camacupa,AO,-12.0167,17.4833,60.80,91,100,1.61,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,74.66,96,100,8.48,light rain
2,2,Boa Vista,BR,2.8197,-60.6733,80.58,61,40,9.22,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.26,74,16,15.68,few clouds
6,6,Salalah,OM,17.0151,54.0924,66.29,63,0,3.44,clear sky
8,8,Port Elizabeth,ZA,-33.9180,25.5701,66.09,93,100,6.53,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

295


City_ID                295
City                   295
Country                291
Lat                    295
Lng                    295
Max Temp               295
Humidity               295
Cloudiness             295
Wind Speed             295
Current Description    295
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

291


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,74.66,96,100,8.48,light rain
2,2,Boa Vista,BR,2.8197,-60.6733,80.58,61,40,9.22,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.26,74,16,15.68,few clouds
6,6,Salalah,OM,17.0151,54.0924,66.29,63,0,3.44,clear sky
8,8,Port Elizabeth,ZA,-33.9180,25.5701,66.09,93,100,6.53,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,74.66,light rain,-22.9661,-42.0278,
2,Boa Vista,BR,80.58,scattered clouds,2.8197,-60.6733,
3,Atuona,PF,78.26,few clouds,-9.8000,-139.0333,
6,Salalah,OM,66.29,clear sky,17.0151,54.0924,
8,Port Elizabeth,ZA,66.09,overcast clouds,-33.9180,25.5701,
9,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,
10,Rikitea,PF,77.41,overcast clouds,-23.1203,-134.9692,
12,Bluff,NZ,71.80,overcast clouds,-46.6000,168.3333,
13,Voh,NC,85.95,overcast clouds,-20.9667,164.7000,
14,San Policarpo,PH,82.76,broken clouds,12.1791,125.5072,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,74.66,light rain,-22.9661,-42.0278,Pousada Porto Praia
2,Boa Vista,BR,80.58,scattered clouds,2.8197,-60.6733,Aipana Plaza Hotel
3,Atuona,PF,78.26,few clouds,-9.8000,-139.0333,Villa Enata
6,Salalah,OM,66.29,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
8,Port Elizabeth,ZA,66.09,overcast clouds,-33.9180,25.5701,39 On Nile Guest House
9,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,Paradise Inn
10,Rikitea,PF,77.41,overcast clouds,-23.1203,-134.9692,People ThankYou
12,Bluff,NZ,71.80,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
13,Voh,NC,85.95,overcast clouds,-20.9667,164.7000,Le gîte du Koniambo
14,San Policarpo,PH,82.76,broken clouds,12.1791,125.5072,Agapita M. Nicart


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))